## Imports

In [1]:
import time
import pyswarms as ps
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from base64 import b64encode
from IPython.display import HTML
plt.style.use('ggplot')
from numba import jit, njit

# jitted

In [2]:
@njit
def S(z: float, phi: np.array([float])) -> float:
  if z <= abs(phi[6]):
    return 0
  if z >= abs(phi[5]):
    return 1
  else:
    return 0.5 - 0.5 * np.tanh(1/(z-abs(phi[5]))+1/(z-abs(phi[6])))


In [3]:
@njit
def Al(v: np.array([float]), phi: np.array([float]), i: int) -> np.array([float]):
  res = np.zeros(int(phi[16]))
  
  for j in range(int(phi[15]+1)):
    if j == i:
      continue
    res += 1/(phi[15]+1) * (abs(phi[2]) / ((1 + np.linalg.norm(v[i]-v[j]) ** 2)) ** abs(phi[4]))* (v[int(phi[15]+1)+j]-v[int(phi[15]+1)+i]) * S(np.dot((v[j]-v[i])/np.linalg.norm(v[i]-v[j]), v[int(phi[15]+1)+i]/np.linalg.norm(v[int(phi[15]+1)+i])), phi)
      
  return res

In [4]:
@jit
def F_0(v: np.array([float]), phi: np.array([float]), i: int) -> np.array([float]):
  if i == 0:
    return np.zeros(int(phi[16]))
#   print('\n(**)i = {}\n phi[3]= {},\n phi[7]= {},\n phi[8]= {},\n phi[9]= {},\n phi[10]= {},\n norm = {},\n exp_7_8 = {},\n exp_9_10 = {},\n vec_uni = {}'.format(
#     i, phi[3], phi[7], phi[8], phi[9], phi[10], np.linalg.norm(v[i]- v[0]) , phi[7]/phi[8] * np.exp(-np.linalg.norm(v[i]- v[0])/phi[8]), phi[9]/phi[10] * np.exp(-np.linalg.norm(v[i]- v[0])/phi[10]), (v[i]- v[0])/np.linalg.norm(v[i]- v[0])))
  
  return - abs(phi[3]) * (v[i]- v[0])/np.linalg.norm(v[i]- v[0]) * (abs(phi[7]/phi[8]) * np.exp(-np.linalg.norm(v[i]- v[0])/abs(phi[8])) - abs(phi[9]/phi[10]) * np.exp(-np.linalg.norm(v[i]- v[0])/abs(phi[10])))

In [5]:
@njit
def F(v: np.array([float]), phi: np.array([float]), i: int) -> np.array([float]):
  res = np.zeros(int(phi[16]))
  
  for j in range(int(phi[15]+1)):
    if j == i:
      continue
    res += -1/(phi[15]+1) * (v[i]- v[j])/np.linalg.norm(v[i]- v[j]) * (abs(phi[11]/phi[12]) * np.exp(-np.linalg.norm(v[i]- v[j])/abs(phi[12])) - abs(phi[13]/phi[14]) * np.exp(-np.linalg.norm(v[i]- v[j])/abs(phi[14])))
      
  return res

In [6]:
@jit
def f(v: np.array([float]), phi: np.array([float])) -> np.array([float]):

  res = np.zeros((int(2*phi[15]+2),int(phi[16])))

  for i in range(int(phi[15]+1)):
#     print('\n(**)i = {}\n phi[0]= {},\n phi[1]= {},\n norm = {},\n v_i = {},\n F_i = {},\n Al = {},\n F_0 = {}'.format(
#     i, phi[0], phi[1], np.linalg.norm(v[int(phi[15]+1)+i]) ** 2, v[int(phi[15]+1)+i], F(v, phi, i), Al(v, phi, i), F_0(v, phi, i)))
    res[i] = v[int(phi[15]+1)+i]
    res[int(phi[15]+1)+i] = (abs(phi[0]) - abs(phi[1]) * np.linalg.norm(v[int(phi[15]+1)+i]) ** 2) * v[int(phi[15]+1)+i] + F(v, phi, i) + Al(v, phi, i) + F_0(v, phi, i)

  return res

In [7]:
def run(q: np.array([float]), phi: np.array([float]), t: int, h: float, save = False, file ='animation') -> np.array([float]):

  r = int( t/h ) 

  res = np.zeros((r+1,4))  

  Q = np.zeros((r+2,int(2*phi[15]+2),int(phi[16])))
  Q[0] = q

  for j in range(r+1):
    K_1 = f(Q[j], phi)
    K_2 = f(Q[j] + h/2 * K_1, phi)
    K_3 = f(Q[j] + h/2 * K_2, phi)
    K_4 = f(Q[j] + h * K_2, phi)
    Q[j+1] = Q[j] + h/6 * (K_1 + 2*K_2 + 2*K_3 + K_4)


  if save :
    
    for j in range(r+1):
    
        res[j, 0] = X_t(Q[j], phi)
        res[j, 1] = V_t(Q[j], phi)
        res[j, 2] = mean_x(Q[j], phi)
        res[j, 3] = mean_v(Q[j], phi)
    
    fig = plt.figure(constrained_layout=True, figsize = (16, 9))
    gs = fig.add_gridspec(4, 3)


    if phi[16] == 2:
      ax0 = fig.add_subplot(gs[:,0:2])

      p01 = ax0.scatter(q[0,0], q[0,1], marker="o", color = "b")
      p02 = ax0.quiver(q[0,0], q[0,1], q[phi[15]+1,0] / (np.sqrt(q[phi[15]+1,0]**2+q[phi[15]+1,1]**2)), q[phi[15]+1,1] / (np.sqrt(q[phi[15]+1,0]**2+q[phi[15]+1,1]**2)), width= 0.001, headwidth=3, headlength=5, color='b')

      p03 = ax0.scatter(q[1:phi[15]+1,0], q[1:phi[15]+1,1], marker="o", color = "r")
      p04 = ax0.quiver(q[1:phi[15]+1,0], q[1:phi[15]+1,1], q[phi[15]+2:,0]/(np.sqrt(q[phi[15]+2:,0]**2+q[phi[15]+2:,1]**2)), q[phi[15]+2:,1]/(np.sqrt(q[phi[15]+2:,0]**2+q[phi[15]+2:,1]**2)), width= 0.001, headwidth=3, headlength=5)
    else:
      ax0 = fig.add_subplot(gs[:,0:2], projection='3d') 
      ax0.set_zlim(Q[:,:phi[15]+1,2].min() - 3,Q[:,:phi[15]+1,2].max() + 3) 

      p01 = ax0.scatter(q[0,0:1], q[0,1:2], q[0,-1], marker="*", color = "b" )
      # nm1 = np.sqrt(q[phi[15]+1,0]**2 + q[phi[15]+1,1]**2 + q[phi[15]+1,2]**2)
      p02 = ax0.quiver(q[0,0], q[0,1], q[0,2], q[phi[15]+1,0], q[phi[15]+1,1], q[phi[15]+1,2], length=1, normalize =True, colors='b', linewidth =1)

      # nm2 = np.sqrt(q[phi[15]+2:,0]**2 + q[phi[15]+2:,1]**2 + q[phi[15]+2:,2]**2)
      p03 = ax0.scatter(q[1:phi[15]+1,0:1], q[1:phi[15]+1,1:2], q[1:phi[15]+1,-1], marker="o", color = "r")
      p04 = ax0.quiver(q[1:phi[15]+1,0], q[1:phi[15]+1,1], q[1:phi[15]+1,2], q[phi[15]+2:,0], q[phi[15]+2:,1], q[phi[15]+2:,2], length=1, normalize =True, colors='k', linewidth =1)

      def quiver_data_to_segments(X, Y, Z, u, v, w, length=1):
        segments = (X, Y, Z, X+v*length, Y+u*length, Z+w*length)
        segments = np.array(segments).reshape(6,-1)
        return [[[x, y, z], [u, v, w]] for x, y, z, u, v, w in zip(*list(segments))]

    ax0.set_xlim(Q[:,:phi[15]+1,0].min() - 5,Q[:,:phi[15]+1,0].max() + 5)
    ax0.set_ylim(Q[:,:phi[15]+1,1].min() - 3,Q[:,:phi[15]+1,1].max() + 3)
      
    ax1 = fig.add_subplot(gs[0, 2])
    p1 = ax1.plot([], [], 'b')[0]

    ax1.set_xlabel("time")
    ax1.set_ylabel("X(t)")
    ax1.set_title('$X(t)$')
    ax1.set_xlim(0,t+1)
    ax1.set_ylim(res[:, 0].min() -2, res[:, 0].max() + 3)

    ax2 = fig.add_subplot(gs[1, 2])
    p2 = ax2.plot([],[], 'b')[0]
    ax2.set_xlim(0,t+1)
    ax2.set_ylim(min(res[:, 1].min(), dx_lim) -2, max(res[:, 1].max(), dx_lim) + 3)
    ax2.set_xlabel("time")
    ax2.set_ylabel("V(t)")
    ax2.set_title('$V(t)$')

    ax3 = fig.add_subplot(gs[2, 2])
    p3 = ax3.plot([], [],'b--', label='$sum(|x_i - x_0|)\;/\;(N+1)$')[0]
    ax3.plot(np.arange(t+1), dx_lim*np.ones(t+1), 'k', label='$ln(C^0_rl^0_a/(l^0_rC^0_a))*(l^0_al^0_r/(l^0_a-l^0_r))$')
    ax3.set_xlim(0,t+1)
    ax3.set_ylim(min(res[:, 2].min(), v_lim) - 2, max(res[:, 2].max(), v_lim) + 3)
    ax3.set_xlabel("time")
    ax3.set_ylabel("Distance")
    ax3.set_title('The distance between the agents and the leader')
    ax3.legend()

    ax4 = fig.add_subplot(gs[3, 2])
    p4 = ax4.plot([], [], 'b--', label='$sum(|v_i|)\;/\;(N+1)$')[0]
    ax4.plot(np.arange(t+1), v_lim*np.ones(t+1), 'k', label='$sqrt((alpha)/(beta))$')
    ax4.set_xlim(0,t+1)
    ax4.set_ylim(res[:, 3].min() - 2, res[:, 3].max() + 3)
    ax4.set_xlabel("time")
    ax4.set_ylabel("mean(|v|)")
    ax4.set_title('The mean norm of velocity')
    ax4.legend()

    def animate(i):  
        time = np.linspace(0,i*h,i+1)

        if phi[16]==2:

          p01.set_offsets(np.c_[Q[i,0,0], Q[i,0,1]])
          
          nrm1 = np.sqrt(Q[i,phi[15]+1,0]**2 + Q[i,phi[15]+1,0]**2)
          p02.set_offsets(np.c_[Q[i,0,0], Q[i,0,1]])
          p02.set_UVC(Q[i,phi[15]+1,0]/nrm1, Q[i,phi[15]+1,1]/nrm1)
          
          p03.set_offsets(np.c_[Q[i,1:phi[15]+1,0], Q[i,1:phi[15]+1,1]])
          
          nrm2 = np.sqrt(Q[i,phi[15]+2:,0]**2 + Q[i,phi[15]+2:,1]**2)
          p04.set_offsets(np.c_[Q[i,1:phi[15]+1,0], Q[i,1:phi[15]+1,1]])
          p04.set_UVC(Q[i,phi[15]+2:,0]/nrm2, Q[i,phi[15]+2:,1]/nrm2)

        else:
          p01._offsets3d = (Q[i,0,0:1], Q[i,0,1:2], Q[i,0,-1])
          
          # nrm1 = np.sqrt(Q[i,phi[15]+1,0]**2 + Q[i,phi[15]+1,0]**2)
          segs1 = quiver_data_to_segments(Q[i,0,0], Q[i,0,1], Q[i,0,2], Q[i,phi[15]+1,0], Q[i,phi[15]+1,1], Q[i,phi[15]+1,2], length=1)
          p02.set_segments(segs1)
          
          p03._offsets3d = (Q[i,1:phi[15]+1,0], Q[i,1:phi[15]+1,1], Q[i,1:phi[15]+1,2])
          
          # nrm2 = np.sqrt(Q[i,phi[15]+2:,0]**2 + Q[i,phi[15]+2:,1]**2)
          segs2 = quiver_data_to_segments(Q[i,1:phi[15]+1,0], Q[i,1:phi[15]+1,1], Q[i,1:phi[15]+1,2], Q[i,phi[15]+2:,0], Q[i,phi[15]+2:,1], Q[i,phi[15]+2:,2], length=1)
          p04.set_segments(segs2)

        p1.set_data(time, res[:i+1, 0])
        p2.set_data(time, res[:i+1, 1])
        p3.set_data(time, res[:i+1, 2])
        p4.set_data(time, res[:i+1, 3])


    anim = FuncAnimation(fig, animate, interval=100, frames=r+1)

    anim.save(f'{file}.gif')

    plt.close()
  return Q 




### Dispersion, disagreement and asymptotic behavior:

In [8]:
def X_t(v, phi):
  res = 0
  for i in range(int(phi[15])+1):
    for j in range(int(phi[15])+1):
      res += np.linalg.norm(v[i] - v[j])  
  return 0.5 / ((phi[15]+1)**2) * res
   

In [9]:
def V_t(v, phi):
  res = 0
  for i in range(int(phi[15])+1):
    for j in range(int(phi[15])+1):
      res += np.linalg.norm(v[int(phi[15])+1+i] - v[j+int(phi[15])+1])  
  return 0.5 / ((phi[15]+1)**2) * res

In [10]:
def mean_x(v, phi):
  res = 0
  for i in range(1,int(phi[15])+1):
    res += np.linalg.norm(v[i] - v[0])

  return (1 / phi[15]) * res 

In [11]:
def mean_v(v, phi):
  res = 0
  for i in range(int(phi[15])+1):
    res += np.linalg.norm(v[int(phi[15])+1+i])

  return (1 / (phi[15]+1)) * res 

# Initialisation 

In [12]:
def init_20(phi):

  global dx_lim, v_lim
  
  
  alpha = phi[0]
  beta = phi[1]

  gamma = phi[2]
  gamma_1 = phi[3]
  sigma = phi[4]

  d1 = phi[5]
  d2 = phi[6]

  C0_a = phi[7]
  l0_a = phi[8]
  C0_r = phi[9]
  l0_r = phi[10]

  C_a = phi[11]
  l_a = phi[12]
  C_r = phi[13]
  l_r = phi[14]

  N = phi[15]
  d = phi[16]


  dx_lim = np.log( (C0_r * l0_a) / (C0_a * l0_r) ) * (l0_a * l0_r) / (l0_a - l0_r)
  v_lim = np.sqrt(alpha/beta)

  res = np.zeros((2 * N + 2, d))

  res[0] = [20, 25]
    
  for i in range(10):
    res[1+i] = [2 * np.cos(np.pi * i / 5), 2 * np.sin(np.pi * i / 5)]
  
  for i in range(10):
    res[11+i] = [15 + 2 * np.cos(np.pi * i / 5), 2 * np.sin(np.pi * i / 5)]
    
  
  x = res[:N+1,:].copy()
    
  tmp = np.ones((21,2))
  x[:,0] = sum(x[:,0]) / (21) * tmp[:,0]
  x[:,1] = sum(x[:,1]) / (21) * tmp[:,1]


  res[N+1:,:] = np.sqrt(alpha/beta) * (res[:N+1,:] - x) / np.linalg.norm(res[:N+1,:])
  
  
  return res

In [13]:
def init_1(phi):

  global dx_lim, v_lim
  
  
  alpha = phi[0]
  beta = phi[1]

  gamma = phi[2]
  gamma_1 = phi[3]
  sigma = phi[4]

  d1 = phi[5]
  d2 = phi[6]

  C0_a = phi[7]
  l0_a = phi[8]
  C0_r = phi[9]
  l0_r = phi[10]

  C_a = phi[11]
  l_a = phi[12]
  C_r = phi[13]
  l_r = phi[14]

  N = phi[15]
  d = phi[16]


  dx_lim = np.log( (C0_r * l0_a) / (C0_a * l0_r) ) * (l0_a * l0_r) / (l0_a - l0_r)
  v_lim = np.sqrt(alpha/beta)

  res = np.zeros((2 * N + 2, d))
  res[0] = [20, 25]
  for i in range(12):
    res[1+i] = [2 * np.cos(np.pi * i / 6), 2 * np.sin(np.pi * i / 6)]
  for i in range(25):
    res[13+i] = [4 * np.cos(np.pi * 2*i / 25), 4 * np.sin(np.pi * 2*i / 25)]
  for i in range(12):
    res[38+i] = [15 + 2 * np.cos(np.pi * i / 6), 2 * np.sin(np.pi * i / 6)]
  res[50] = [15,0]
  for i in range(25):
    res[51+i] = [15 + 4 * np.cos(np.pi * 2*i / 25), 4 * np.sin(np.pi * 2*i / 25)]
  x = res[:N+1,:].copy()
  tmp = np.ones((76,2))
  x[:,0] = sum(x[:,0]) / (76) * tmp[:,0]
  x[:,1] = sum(x[:,1]) / (76) * tmp[:,1]


  res[N+1:,:] = np.sqrt(alpha/beta) * (res[:N+1,:] - x) / np.linalg.norm(res[:N+1,:])
  
  
  return res

# Plot functions

In [14]:
def plot_sys(v, phi):

  if phi[16] == 2:
    lx = v[0, 0]
    ly = v[0, 1]
    
    lvx = v[phi[15]+1,0]
    lvy = v[phi[15]+1,1]

    x = v[1:phi[15]+1,0]
    y = v[1:phi[15]+1,1]

    vx = v[phi[15]+2:,0]
    vy = v[phi[15]+2:,1]

    fig, ax = plt.subplots(figsize = (10,10))
    plt.xlim(min(min(x), lx) - 5,max(max(x), lx) + 5)
    plt.ylim(min(min(y), ly) - 3,max(max(y), ly) + 3)

    plt.scatter(lx, ly, marker="o", color = "b")
    plt.quiver(lx, ly, lvx, lvy, scale = 1, width= 0.001, headwidth=3, headlength=5, color='b')

    plt.scatter(x, y, marker="o", color = "r")
    plt.quiver(x, y, vx, vy, scale = 1, width= 0.001, headwidth=3, headlength=5)

    plt.show()

  else:
    lx = v[0, 0]
    ly = v[0, 1]
    lz = v[0, 2]
    
    lvx = v[phi[15]+1,0]
    lvy = v[phi[15]+1,1]
    lvz = v[phi[15]+1,2]

    x = v[1:phi[15]+1,0]
    y = v[1:phi[15]+1,1]
    z = v[1:phi[15]+1,2]

    vx = v[phi[15]+2:,0]
    vy = v[phi[15]+2:,1]
    vz = v[phi[15]+2:,2]

    fig = plt.figure(figsize=(12,10))
    ax = fig.add_subplot(111, projection='3d')

    ax.set_xlim(min(min(x), lx) - 5,max(max(x), lx) + 5)
    ax.set_ylim(min(min(y), ly) - 5,max(max(y), ly) + 5)
    ax.set_ylim(min(min(z), lz) - 5,max(max(z), lz) + 5)

    ax.scatter(lx, ly, lz, marker="*", color = "b" )
    ax.quiver(lx, ly, lz, lvx, lvy, lvz, length=3, colors='b', linewidth =1)

    ax.scatter(x, y, z, marker="o", color = "r")
    ax.quiver(x, y, z, vx, vy, vz, length=1, colors='k', linewidth =1)

    ax.set_xlabel('x axis')
    ax.set_ylabel('y axis')
    ax.set_zlabel('z axis')


    plt.show()

In [15]:
def plot_asym_behavior(w, t):

  r = np.arange(t+1)

  fig = plt.figure(constrained_layout=True, figsize = (16, 9))
  gs = fig.add_gridspec(2, 2)

  f_ax1 = fig.add_subplot(gs[0, 0])
  f_ax1.plot(r, w[:, 0])
  f_ax1.set_xlabel("time")
  f_ax1.set_ylabel("X(t)")
  f_ax1.set_title('$X(t)$')


  f_ax2 = fig.add_subplot(gs[0, 1])
  f_ax2.plot(r, w[:, 1])
  f_ax2.set_xlabel("time")
  f_ax2.set_ylabel("V(t)")
  f_ax2.set_title('$V(t)$')


  f_ax3 = fig.add_subplot(gs[1, 0])
  f_ax3.plot(r, w[:, 2], '--', label='$sum(|x_i - x_0|)\;/\;(N+1)$')
  f_ax3.plot(r, dx_lim * np.ones(t+1), 'k', label='$ln(C^0_rl^0_a/(l^0_rC^0_a))*(l^0_al^0_r/(l^0_a-l^0_r))$')
  f_ax3.set_xlabel("time")
  f_ax3.set_ylabel("Distance")
  f_ax3.set_title('The distance between the agents and the leader')
  f_ax3.legend()

  f_ax4 = fig.add_subplot(gs[1, 1])
  f_ax4.plot(r, w[:, 3], '--', label='$sum(|v_i|)\;/\;(N+1)$')
  f_ax4.plot(r, v_lim * np.ones(t+1), 'k', label='$sqrt((alpha)/(beta))$')
  f_ax4.set_xlabel("time")
  f_ax4.set_ylabel("mean(|v|)")
  f_ax4.set_title('The mean norm of velocity')
  f_ax4.legend()

# parameters

In [16]:
params_1 = (0.07 , 0.05 , 1 , 10 , 0.5 , np.cos(1.047) , np.cos(1.57), 10 , 50 , 25 , 1 , 20 , 100 , 50 , 2, 75, 2)

params_20 = (0.07 , 0.05 , 1 , 10 , 0.5 , np.cos(1.047) , np.cos(1.57), 10 , 50 , 25 , 1 , 20 , 100 , 50 , 2, 20, 2)

params_2 = (0.07, 0.05, 1, 10, 0.5, np.cos(1.047), np.cos(1.57), 10, 50, 25, 1, 50, 10, 20, 2, 61, 2)

# Parameter identification

In [23]:

def objective(phi: np.array):
  res = np.zeros(phi.shape[0])
  for k in range(phi.shape[0]):
#     phi[k][5:] = params_20[5:]    
    phi[k][15:] = params_20[15:]
    
    temp = run(Y, phi[k], 20, 0.5)

    res[k] = np.linalg.norm(Y_data[:-1] - temp[:-1]) ** 2

  return res 

In [24]:
# params = (alpha, beta, gamma, gamma_1, sigma, d1, d2, C0_a, l0_a, C0_r, l0_r, C_a, l_a, C_r, l_r, N, d)

In [25]:
Y = init_20(params_20)

In [26]:
Y_data = run(Y, params_20, 20, 0.5)

In [28]:
%%time

# Set-up hyperparameters
options = {'c1': 0.5, 'c2': 0.8, 'w':0.9}

a=1

# Create bounds
min_bound = np.array([0, 0, 0, 0, 0, -1, -1, 0, 50 - a , 25 - a , 0, 20 - a , 100 - a , 50 - a , 0, 0, 0])

# max_bound = 100*np.ones(17)
max_bound = np.array([0.07 + 1 , 0.05 + 1 , 1 + a , 10 + a , 0.5 + a , 1, 1, 10 + a , 50 + a , 25 + a , 1 + a , 20 + a , 100 + a , 50 + a , 2 + a, 20 + a, 2 + a])
bounds = (min_bound, max_bound)

# Call instance of GlobalBestPSO
optimizer_3 = ps.single.GlobalBestPSO(n_particles=24, dimensions=17,
                                    options=options)

# Perform optimization
stats = optimizer_3.optimize(objective, iters=3000, n_processes=8)

2021-06-18 06:25:05,762 - pyswarms.single.global_best - INFO - Optimize for 3000 iters with {'c1': 0.5, 'c2': 0.8, 'w': 0.9}
pyswarms.single.global_best:   1%|          |28/3000, best_cost=2.23e+4<ipython-input-7-62c1fbf4fe07>:15: RuntimeWarning: overflow encountered in multiply
  Q[j+1] = Q[j] + h/6 * (K_1 + 2*K_2 + 2*K_3 + K_4)
pyswarms.single.global_best: 100%|██████████|3000/3000, best_cost=1.19e+3
2021-06-18 06:57:44,785 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1189.7074569157592, best pos: [ 4.56648729e-07  4.71637301e-02 -2.86913090e+00  1.37898190e+01
  1.45711035e+01  6.79473297e-01  1.18658205e+01  8.63900597e+00
  5.74039884e+01 -7.42514358e+01  9.41441247e-01  3.95030466e+01
 -1.01921846e+00  7.23303131e+01  1.69536069e+00  2.67047082e+00
  2.28567262e+01]


CPU times: user 30.6 s, sys: 4.51 s, total: 35.1 s
Wall time: 32min 39s


In [43]:
%%time

# Set-up hyperparameters
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

a=1

# Create bounds
min_bound = np.array([0, 0, 0, 0, 0, -1, -1, 0, 50 - a , 25 - a , 0, 20 - a , 100 - a , 50 - a , 0, 0, 0])

# max_bound = 100*np.ones(17)
max_bound = np.array([0.07 + 1 , 0.05 + 1 , 1 + a , 10 + a , 0.5 + a , 1, 1, 10 + a , 50 + a , 25 + a , 1 + a , 20 + a , 100 + a , 50 + a , 2 + a, 20 + a, 2 + a])
bounds = (min_bound, max_bound)

# Call instance of GlobalBestPSO
optimizer_7 = ps.single.GlobalBestPSO(n_particles=24, dimensions=17,
                                    options=options, init_pos=optimizer_7.pos_history[-1])

# Perform optimization
stats = optimizer_7.optimize(objective, iters=1000, n_processes=8)

2021-06-17 19:22:59,944 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=7.83e+6
2021-06-17 19:48:52,860 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 7825572.279350761, best pos: [-0.39365049 -0.39069636  1.21320004 -0.26491914  2.12999627  0.81940542
  0.33588485  0.46187302 -0.42032737 -1.33811356  3.571347    7.39523374
  1.49882603  1.89089711  4.25468284  3.77601874  0.4741483 ]


CPU times: user 11.3 s, sys: 1.81 s, total: 13.1 s
Wall time: 25min 52s


In [44]:
%%time

# Set-up hyperparameters
options = {'c1': 0.6, 'c2': 0.9, 'w':0.9}

a=1

# Create bounds
min_bound = np.array([0, 0, 0, 0, 0, -1, -1, 0, 50 - a , 25 - a , 0, 20 - a , 100 - a , 50 - a , 0, 0, 0])

# max_bound = 100*np.ones(17)
max_bound = np.array([0.07 + 1 , 0.05 + 1 , 1 + a , 10 + a , 0.5 + a , 1, 1, 10 + a , 50 + a , 25 + a , 1 + a , 20 + a , 100 + a , 50 + a , 2 + a, 20 + a, 2 + a])
bounds = (min_bound, max_bound)

# Call instance of GlobalBestPSO
optimizer_8 = ps.single.GlobalBestPSO(n_particles=24, dimensions=17,
                                    options=options)

# Perform optimization
stats = optimizer_8.optimize(objective, iters=1000, n_processes=8)

2021-06-18 05:49:31,259 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.6, 'c2': 0.9, 'w': 0.9}
pyswarms.single.global_best:  28%|██▊       |277/1000, best_cost=8.89e+6<ipython-input-7-62c1fbf4fe07>:15: RuntimeWarning: overflow encountered in multiply
  Q[j+1] = Q[j] + h/6 * (K_1 + 2*K_2 + 2*K_3 + K_4)
<ipython-input-7-62c1fbf4fe07>:15: RuntimeWarning: invalid value encountered in add
  Q[j+1] = Q[j] + h/6 * (K_1 + 2*K_2 + 2*K_3 + K_4)
pyswarms.single.global_best:  34%|███▍      |339/1000, best_cost=8.86e+6Process ForkPoolWorker-137:
Process ForkPoolWorker-144:
Process ForkPoolWorker-141:
Process ForkPoolWorker-138:
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-142:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-143:
Process ForkPoolWorker-139:
  Fi

KeyboardInterrupt: 

KeyboardInterrupt
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-31-010ecb9ccf7f>", line 7, in objective
    temp = run(Y, phi[k], 50, 0.5)
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "<ipython-input-31-010ecb9ccf7f>", line 7, in objective
    temp = run(Y, phi[k], 50, 0.5)
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "<ipython-input-7-62c1fbf4fe07>", line 14, in run
    K_4 = f(Q[j] + h * K_2, phi)
  File "<ipython-input-7-62c1fbf4fe07>", line 11, in run
    K_1

In [77]:
init = np.array([[0.06284261,  0.05206153,  5.81268423, 10.06835564,  4.90229861,  2.71729476, 6.13536684, 10 , 50 , 25 , 1 , 20 , 100 , 50 , 2, 20, 2] for _ in range(40)])

In [29]:
# plt.plot(optimizer.cost_history)
# objective(optimizer.pos_history[-2])
run(Y,params_pred3,100,0.5, save=True, file="test_pred3")
# optimizer.pos_history[-1]

2021-06-16 18:30:14,376 - matplotlib.animation - WARNING - MovieWriter ffmpeg unavailable; using Pillow instead.
2021-06-16 18:30:14,377 - matplotlib.animation - INFO - Animation.save using <class 'matplotlib.animation.PillowWriter'>


array([[[ 2.00000000e+01,  2.50000000e+01],
        [ 2.00000000e+00,  0.00000000e+00],
        [ 1.61803399e+00,  1.17557050e+00],
        ...,
        [ 1.28422885e-01, -6.31742353e-02],
        [ 1.53672811e-01, -6.31742353e-02],
        [ 1.74100430e-01, -4.83327011e-02]],

       [[ 2.01159290e+01,  2.52307232e+01],
        [ 2.23078858e+00,  9.99245012e-02],
        [ 1.81058328e+00,  1.40685134e+00],
        ...,
        [-2.42173294e-02, -2.38161272e-01],
        [ 4.69246014e-01, -2.26197099e-01],
        [ 8.59381848e-01,  8.83489742e-02]],

       [[ 2.02205037e+01,  2.54364438e+01],
        [ 2.96345505e+00,  4.19500613e-01],
        [ 2.44891416e+00,  2.04563044e+00],
        ...,
        [-1.20822193e-01, -2.38916870e-01],
        [ 7.32140993e-01, -2.04255917e-01],
        [ 1.36690072e+00,  3.47884880e-01]],

       ...,

       [[ 6.66126306e+01,  1.77293871e+02],
        [ 6.27780283e+01,  1.81075199e+02],
        [ 7.04503510e+01,  1.73479131e+02],
        ...,
     

In [28]:
params_20   = ( 0.07 ,           0.05 ,           1 ,              10 , 
                0.5 ,            np.cos(1.047) ,  np.cos(1.57),    10 ,
                50 ,             25 ,             1 ,              20 ,
                100 ,            50 ,             2,               20,
                2)

params_pred = ( 6.22712065e-02,  5.06104633e-02,  4.55374230e+00,  1.52257329e+01,
                1.31434289e+00,  2.10709514e+00,  3.15172943e+00,  6.65762399e+00,
                5.06755363e+01,  2.49981224e+01,  9.86426738e-01,  1.84465740e+01,
                1.04021877e+02,  4.99368921e+01,  1.97556702e+00,  20,
                2)
params_pred2 = ( 2.39064359e-01, 7.68831092e-02, 9.37504712e+00, 2.24346935e+01,
                 1.06140789e+01, 2.12767063e-02, 2.26398337e-01, 4.17479246e+00,
                 4.65354086e+01, 2.25497370e+01, 1.94129444e+00, 2.64774279e+01,
                 9.88482868e+01, 5.05689676e+01, 1.79416162e+00, 20,
                 2)

params_pred3 = (  4.61665940e-02,  4.72920198e-02,  1.77824176e+00,  3.27155919e+00,
                  6.69389722e-01,  5.62934398e-01,  9.44237439e-02,  3.17047517e+01,
                  5.30360005e+01,  5.81683321e+01,  1.09202426e+00,  2.92999387e+01,
                  1.82094659e+02,  5.04491220e+01,  1.94084459e+00,  20,
                  2)

# test

In [37]:
objective(np.array([[ 6.75089399e-02,  4.83759348e-02,  2.97404484e+00,  4.49835368e+00,
 -2.18877975e-01,np.cos(1.047), np.cos(1.57), 2.21491142e+01,  4.94508429e+01,  2.54305379e+00,
  3.15316226e+00, -3.02071074e+00,  3.07831543e+00,  2.32741483e+01,
  4.73761734e-01, 75, 2]]))

# 131.14808533

array([131.14808533])

In [30]:
objective(np.array([[2.85847473e-02, 4.52504452e-02, 2.01671110e-01 ,1.02330002e+01 , 0.5, np.cos(1.047), np.cos(1.57), 10 , 50 , 25 , 1, 20 , 100 , 50 , 2 , 75, 2 ]]))

array([0.05067263])

In [ ]:
a = [1, 2, 3, 4, 5]
a[4:]